In [1]:
from brian2 import *
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

c:\Users\Thorben\AppData\Local\Programs\Python\Python312\Lib\site-packages\brian2\utils\logger.py:23: UserWarning: A NumPy version >=1.22.4 and <2.3.0 is required for this version of SciPy (detected version 2.4.1)
  import scipy


In [2]:
def run_w_startvector(runtime, startvector):   #enter the runtime in seconds but with the unit
    
    start_scope()
    # Brian Zeitschritt (bei kontinuierlicher Simulation)

    defaultclock.dt = 0.1*ms

    # Simulationsdauer
    


    # --- Parameter ---
    N       = 10_000  # 10_000
    K       = 1_000   # 1_000

    V_T     = 1.0
    V_R     = 0.0

    nu      = 10*Hz
    J_0     = 1.0
    tau_m   = 10*ms




        # balanced-state prediction (eq. (5)): nu ~ nu_bal = I_0/(J_0*tau_m)  -> I_0 ~ nu*J_0*tau_m
    I_0 = nu * J_0 * tau_m

    Iext_const = np.sqrt(K) * I_0  # These inputs counteract the constant excitatory current
    w_inh = J_0/np.sqrt(K)  # neurons receive exclusively inhibitory recurrent inputs
    # NOTE S = Synapses(G, G, on_pre="V -= w_inh")  # inhibitory undelayed pulse 
                


    # --- Neuronmodell (Voltage-LIF) ---
    # Paper: tau_m dV/dt = -V + I(t) (1), mit I(t)=sqrt(K)*I_0 - (J_0/sqrt(K))*sum delta(...)
    # In Brian: wir modellieren I_ext als konstanten Term im ODE und synaptische Pulse als Sprung in V.
    eqs = """
    dV/dt = (-V + I_ext)/tau_m : 1  # NOTE dimensionslos (V normiert und dimensionslos im Paper)
    I_ext : 1                       # NOTE dimensionslos
    """

    G = NeuronGroup(
        N,
        model=eqs,
        threshold="V >= V_T",
        reset="V = V_R",
        method="euler"
    )

    # Initialisierung
    G.V = startvector    # TODO: alternative Initialisierung?
    G.I_ext = Iext_const    # REVIEW: Ich habe aus der Übung nicht verstanden, wie wir die Delta-Funktion hinbekommen..


    # --- Konnektivität: gerichteter Erdős–Rényi mit mittlerem In-degree K ---
    p = K / (N-1)   # p in (0,1)


    print(f"Konnektivität-Setup:")
    print(f"  N = {N}, K = {K}, p = {p:.6f}")
    print(f"  p ~ K/N = {K/N:.3f} (sollte ~0.1 sein für 'sparse')")


    seed(0)           # Brian2 seed
    S = Synapses(G, G, on_pre="V -= w_inh")  # inhibitory undelayed pulse 
    S.connect(condition='i!=j', p=p)


    
    # %%
    # **DIAGNOSE: Realisierter mittlerer In-Degree nach connect()**
    # Zähle für jeden Neuron, wie viele präsynaptische Eingänge es hat
    indegrees = np.bincount(S.j, minlength=N)  # S.j sind die postsynaptischen Neuronen-Indizes
    
    realized_mean_indegree = np.mean(indegrees)
    realized_std_indegree = np.std(indegrees)
    '''
    print(f"\nRealisierte Konnektivität nach S.connect(p={p:.6f}):")
    print(f"  Mittlerer realisierter In-Degree: {realized_mean_indegree:.2f} (erwartet: {K})")
    print(f"  Std-Abweichung: {realized_std_indegree:.2f}")
    print(f"  Min In-Degree: {np.min(indegrees)}, Max In-Degree: {np.max(indegrees)}")
    print(f"  Gesamtzahl Synapsen: {len(S)}")
    '''

    # Warnung falls stark abweichend
    if abs(realized_mean_indegree - K) > 0.1*K:
        print(f"\n⚠️  WARNING: realisierter In-Degree weicht >10% ab!")


    # --- Monitore ---
    # Rasterplot: 30 zufällige Neuronen 
    n_plot = 30
    np.random.seed(0)
    idx = np.random.choice(N, size=n_plot, replace=False)


    sp_mon = SpikeMonitor(G)


    # Spannung eines Neurons (und dessen spikes)
    i_0 = int(idx[0])  # ein Beispielneuron aus dem subset
    #mon_endvalues = StateMonitor(G, "V", record=True, dt=runtime)


    # --- Run ---
    print(f"\nRunning simulation for T = {runtime}...")

    # Set BOTH seeds right after start_scope()

    run(runtime)
    mon_endvalues = np.array(G.V)
    print("Done!")

    return mon_endvalues



In [3]:
# create an initial random vector to have a random starting point for the simulation
initial_randvector = np.random.rand(10000)
# run it for 8 seconds and return the resulting voltage
runtime = 8*second
starting_vector  = run_w_startvector(runtime, initial_randvector)


#run it again but now from the resulting vector of the previous run
runtime = 5*second
vector1  = run_w_startvector(runtime, starting_vector)
vector2  = run_w_startvector(runtime, starting_vector)

runtime = 4*second
vector3 = run_w_startvector(runtime, starting_vector)

Konnektivität-Setup:
  N = 10000, K = 1000, p = 0.100010
  p ~ K/N = 0.100 (sollte ~0.1 sein für 'sparse')


WARNING    Cannot use Cython, a test compilation failed: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/ (DistutilsPlatformError) [brian2.codegen.runtime.cython_rt.cython_rt.failed_compile_test]
INFO       Cannot use compiled code, falling back to the numpy code generation target. Note that this will likely be slower than using compiled code. Set the code generation to numpy manually to avoid this message:
prefs.codegen.target = "numpy" [brian2.devices.device.codegen_fallback]



Running simulation for T = 8. s...
Done!
Konnektivität-Setup:
  N = 10000, K = 1000, p = 0.100010
  p ~ K/N = 0.100 (sollte ~0.1 sein für 'sparse')

Running simulation for T = 5. s...
Done!
Konnektivität-Setup:
  N = 10000, K = 1000, p = 0.100010
  p ~ K/N = 0.100 (sollte ~0.1 sein für 'sparse')

Running simulation for T = 5. s...
Done!
Konnektivität-Setup:
  N = 10000, K = 1000, p = 0.100010
  p ~ K/N = 0.100 (sollte ~0.1 sein für 'sparse')

Running simulation for T = 4. s...
Done!


In [4]:
#np.array_equal(starting_vector0.V.squeeze(), starting_vector7.V.squeeze())
print(starting_vector)
print(vector1)
print(vector2)
print(vector3)

print(np.array_equal(starting_vector, vector1))
print(np.array_equal(vector1, vector2))
print(np.array_equal(vector2, vector3))

[ 0.38701134  0.60162731  0.79538505 ...  0.2214397   0.84397468
 -0.02198302]
[0.5189431  0.56972715 0.47595123 ... 0.30910391 0.54375815 0.43562195]
[0.5189431  0.56972715 0.47595123 ... 0.30910391 0.54375815 0.43562195]
[0.43459245 0.57756312 0.78967413 ... 0.6409965  0.35978837 0.71480179]
False
True
False
